# Summary

+ Baseline  
https://dacon.io/competitions/official/235805/codeshare/3620?page=1&dtype=recent  
이전 다른 이미지 대회에서 수상했던 코드를 참고하였습니다.  
좋은 코드 공유 감사합니다.

+ Data Augmentation   
데이터 불균형이 심해서 학습 이전에 Anomaly한 데이터만 다양한 방법으로 증강하여 새로운 데이터 셋 폴더에 저장하고,  
csv 파일에도 추가한 데이터에 대한 정보를 합쳐서 새로 만들어 주었습니다.  
그리고, 학습 과정에서도 pytorch transforms을 활용하여 데이터를 증강시켜주었습니다.

+ Training parameter
    + Lamb Optimizer
    + ReduceLROnplateau Scheduler
    + WarmUpLR Scheduler
    + Focal Loss
    + Early Stopping


+ Model  
모델은 5 k-fold로 진행하면서 Public 기준으로 결과가 좋은 상위 2개 모델을 골라서 앙상블을 진행해주었습니다.   
그리고, 2가지 모델에 대해서 network 구조와, transforms 방법을 조금 다르게 설정 하였습니다. 다른 모델은 주석으로 표시해두었습니다.
    + cait_s36_384, img_size: 384
    + tf_efficientnet_b7_ns, img_size: 600

+ TTA  
https://github.com/qubvel/ttach  
d4_transform()

# Library

In [1]:
import os
import cv2
import time
import random
import logging
import easydict
import numpy as np
import pandas as pd
from tqdm import tqdm
from os.path import join as opj
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from PIL import Image
from natsort import natsorted

import timm
import torch
import torch.nn as nn
import torch_optimizer as optim
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, grad_scaler
from torchvision import transforms
from torch import Tensor
from torchvision.transforms import functional as F
import torch.cuda.amp as amp
from adamp import AdamP, SGDP

import warnings
warnings.filterwarnings('ignore')

C:\Users\user\.conda\envs\new_dacon\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data Augmentation

train 폴더를 train_add_data 이름으로 새로운 폴더에 복사 후 진행

In [2]:
DATA_DIR = './data'

train_df = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
print(train_df)

                 img_path  label description
0      ./train\가구수정\0.png      0        가구수정
1      ./train\가구수정\1.png      0        가구수정
2     ./train\가구수정\10.png      0        가구수정
3     ./train\가구수정\11.png      0        가구수정
4      ./train\가구수정\2.png      0        가구수정
...                   ...    ...         ...
3452   ./train\훼손\995.png     18          훼손
3453   ./train\훼손\996.png     18          훼손
3454   ./train\훼손\997.png     18          훼손
3455   ./train\훼손\998.png     18          훼손
3456   ./train\훼손\999.png     18          훼손

[3457 rows x 3 columns]


In [3]:
seed = 41
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)

# Random_Crop

In [4]:
def get_random_crop(image, crop_height, crop_width):

    max_x = image.shape[1] - crop_width
    max_y = image.shape[0] - crop_height

    x = np.random.randint(0, max_x)
    y = np.random.randint(0, max_y)

    crop = image[y: y + crop_height, x: x + crop_width]

    return crop

In [5]:
oslabel = list(train_df['label'].unique())
short_label = [0,4,5,8,9,12,13,14,16,17]

for label in tqdm(oslabel):
    for i in range(len(short_label)):
        if short_label[i] == label:
            idx = 0
            one_sample = train_df[train_df['label'] == label].reset_index(drop=True)
            images_list = natsorted(one_sample['img_path'])
            print(images_list)
            for _, image_name in enumerate(images_list):
                image = np.array(Image.open(opj(image_name)).convert('RGB'))

                aug_img = cv2.resize(image, dsize=(1024, 1024))
                aug_img = get_random_crop(aug_img, 900, 900)
                aug_img = cv2.cvtColor(aug_img, cv2.COLOR_BGR2RGB)
                save_path = opj('./data/train_add_data', f'{label}_{idx}_crop.png')
                save_name = f'{label}_{idx}_crop.png'
                idx += 1
                cv2.imwrite(save_path, aug_img)
                train_df.loc[len(train_df)] = [save_path] + one_sample.iloc[0][1:].values.tolist()

  0%|                                                                                           | 0/19 [00:00<?, ?it/s]

['./train\\가구수정\\0.png', './train\\가구수정\\1.png', './train\\가구수정\\2.png', './train\\가구수정\\3.png', './train\\가구수정\\4.png', './train\\가구수정\\5.png', './train\\가구수정\\6.png', './train\\가구수정\\7.png', './train\\가구수정\\8.png', './train\\가구수정\\9.png', './train\\가구수정\\10.png', './train\\가구수정\\11.png']


  5%|████▎                                                                              | 1/19 [00:00<00:07,  2.32it/s]

['./train\\녹오염\\0.png', './train\\녹오염\\1.png', './train\\녹오염\\2.png', './train\\녹오염\\3.png', './train\\녹오염\\4.png', './train\\녹오염\\5.png', './train\\녹오염\\6.png', './train\\녹오염\\7.png', './train\\녹오염\\8.png', './train\\녹오염\\9.png', './train\\녹오염\\10.png', './train\\녹오염\\11.png', './train\\녹오염\\12.png', './train\\녹오염\\13.png']


 26%|█████████████████████▊                                                             | 5/19 [00:00<00:02,  5.41it/s]

...

 79%|████████████████████████████████████████████████████████████████▋                 | 15/19 [00:07<00:02,  1.50it/s]

['./train\\틈새과다\\0.png', './train\\틈새과다\\1.png', './train\\틈새과다\\2.png', './train\\틈새과다\\3.png', './train\\틈새과다\\4.png']


 89%|█████████████████████████████████████████████████████████████████████████▎        | 17/19 [00:08<00:00,  2.18it/s]

['./train\\피스\\0.png', './train\\피스\\1.png', './train\\피스\\2.png', './train\\피스\\3.png', './train\\피스\\4.png', './train\\피스\\5.png', './train\\피스\\6.png', './train\\피스\\7.png', './train\\피스\\8.png', './train\\피스\\9.png', './train\\피스\\10.png', './train\\피스\\11.png', './train\\피스\\12.png', './train\\피스\\13.png', './train\\피스\\14.png', './train\\피스\\15.png', './train\\피스\\16.png', './train\\피스\\17.png', './train\\피스\\18.png', './train\\피스\\19.png', './train\\피스\\20.png', './train\\피스\\21.png', './train\\피스\\22.png', './train\\피스\\23.png', './train\\피스\\24.png', './train\\피스\\25.png', './train\\피스\\26.png', './train\\피스\\27.png', './train\\피스\\28.png', './train\\피스\\29.png', './train\\피스\\30.png', './train\\피스\\31.png', './train\\피스\\32.png', './train\\피스\\33.png', './train\\피스\\34.png', './train\\피스\\35.png', './train\\피스\\36.png', './train\\피스\\37.png', './train\\피스\\38.png', './train\\피스\\39.png', './train\\피스\\40.png', './train\\피스\\41.png', './train\\피스\\42.png', './train\\피스\\43.png

100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [00:10<00:00,  1.89it/s]


# Rotation

In [6]:
def rotation(img, angle):
    angle = int(random.uniform(-angle, angle))
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((int(w/2), int(h/2)), angle, 1)
    img = cv2.warpAffine(img, M, (w, h)) 
    return img

In [7]:
oslabel = list(train_df['label'].unique())
short_label = [0,4,5,8,9,12,13,14,16,17]
# print(train_df)

for label in tqdm(oslabel):
    for i in range(len(short_label)):
        if short_label[i] == label:
            idx = 0
            one_sample = train_df[train_df['label'] == label].reset_index(drop=True)
            print(one_sample)
            print(one_sample.iloc[0][1:].values.tolist() )
            images_list = natsorted(one_sample['img_path'])
#             print(images_list)
            for _, image_name in enumerate(images_list):
                image = np.array(Image.open(opj(image_name)).convert('RGB'))

                aug_img = rotation(image, 30)  
                aug_img = cv2.resize(aug_img, dsize=(1024, 1024))
                aug_img = cv2.cvtColor(aug_img, cv2.COLOR_BGR2RGB)
                save_path = opj('./data/train_add_data', f'{label}_{idx}_rotation.png')
                save_name = f'{label}_{idx}_rotation.png'
                idx += 1
                cv2.imwrite(save_path, aug_img)
                train_df.loc[len(train_df)] = [save_path] + one_sample.iloc[0][1:].values.tolist() 

  0%|                                                                                           | 0/19 [00:00<?, ?it/s]

                               img_path  label description
0                    ./train\가구수정\0.png      0        가구수정
1                    ./train\가구수정\1.png      0        가구수정
2                   ./train\가구수정\10.png      0        가구수정
3                   ./train\가구수정\11.png      0        가구수정
4                    ./train\가구수정\2.png      0        가구수정
5                    ./train\가구수정\3.png      0        가구수정
6                    ./train\가구수정\4.png      0        가구수정
7                    ./train\가구수정\5.png      0        가구수정
8                    ./train\가구수정\6.png      0        가구수정
9                    ./train\가구수정\7.png      0        가구수정
10                   ./train\가구수정\8.png      0        가구수정
11                   ./train\가구수정\9.png      0        가구수정
12   ./data/train_add_data\0_0_crop.png      0        가구수정
13   ./data/train_add_data\0_1_crop.png      0        가구수정
14   ./data/train_add_data\0_2_crop.png      0        가구수정
15   ./data/train_add_data\0_3_crop.png      0        가구

  5%|████▎                                                                              | 1/19 [00:01<00:19,  1.07s/it]

                               img_path  label description
0                     ./train\녹오염\0.png      4         녹오염
1                     ./train\녹오염\1.png      4         녹오염
2                    ./train\녹오염\10.png      4         녹오염
3                    ./train\녹오염\11.png      4         녹오염
4                    ./train\녹오염\12.png      4         녹오염
5                    ./train\녹오염\13.png      4         녹오염
6                     ./train\녹오염\2.png      4         녹오염
7                     ./train\녹오염\3.png      4         녹오염
8                     ./train\녹오염\4.png      4         녹오염
9                     ./train\녹오염\5.png      4         녹오염
10                    ./train\녹오염\6.png      4         녹오염
11                    ./train\녹오염\7.png      4         녹오염
12                    ./train\녹오염\8.png      4         녹오염
13                    ./train\녹오염\9.png      4         녹오염
14   ./data/train_add_data\4_0_crop.png      4         녹오염
15   ./data/train_add_data\4_1_crop.png      4         녹

 26%|█████████████████████▊                                                             | 5/19 [00:02<00:06,  2.17it/s]

...

 79%|████████████████████████████████████████████████████████████████▋                 | 15/19 [00:19<00:06,  1.60s/it]

                              img_path  label description
0                   ./train\틈새과다\0.png     16        틈새과다
1                   ./train\틈새과다\1.png     16        틈새과다
2                   ./train\틈새과다\2.png     16        틈새과다
3                   ./train\틈새과다\3.png     16        틈새과다
4                   ./train\틈새과다\4.png     16        틈새과다
5  ./data/train_add_data\16_0_crop.png     16        틈새과다
6  ./data/train_add_data\16_1_crop.png     16        틈새과다
7  ./data/train_add_data\16_2_crop.png     16        틈새과다
8  ./data/train_add_data\16_3_crop.png     16        틈새과다
9  ./data/train_add_data\16_4_crop.png     16        틈새과다
[16, '틈새과다']


 89%|█████████████████████████████████████████████████████████████████████████▎        | 17/19 [00:19<00:02,  1.11s/it]

                                 img_path  label description
0                        ./train\피스\0.png     17          피스
1                        ./train\피스\1.png     17          피스
2                       ./train\피스\10.png     17          피스
3                       ./train\피스\11.png     17          피스
4                       ./train\피스\12.png     17          피스
..                                    ...    ...         ...
97   ./data/train_add_data\17_46_crop.png     17          피스
98   ./data/train_add_data\17_47_crop.png     17          피스
99   ./data/train_add_data\17_48_crop.png     17          피스
100  ./data/train_add_data\17_49_crop.png     17          피스
101  ./data/train_add_data\17_50_crop.png     17          피스

[102 rows x 3 columns]
[17, '피스']


100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [00:25<00:00,  1.32s/it]


# Flip

In [8]:
for label in tqdm(oslabel):
    for i in range(len(short_label)):
        if short_label[i] == label:
            idx = 0
            one_sample = train_df[train_df['label'] == label].reset_index(drop=True)
            images_list = natsorted(one_sample['img_path'])
            print(images_list)
            for _, image_name in enumerate(images_list):
                image = np.array(Image.open(opj(image_name)).convert('RGB'))

                aug_img = cv2.flip(image, 1)
                aug_img = cv2.resize(aug_img, dsize=(1024, 1024))
                aug_img = cv2.cvtColor(aug_img, cv2.COLOR_BGR2RGB)
                save_path = opj('./data/train_add_data', f'{label}_{idx}_flip1.png')
                save_name = f'{label}_{idx}_flip1.png'
                idx += 1
                cv2.imwrite(save_path, aug_img)
                train_df.loc[len(train_df)] = [save_path] + one_sample.iloc[0][1:].values.tolist()   

  0%|                                                                                           | 0/19 [00:00<?, ?it/s]

['./data/train_add_data\\0_0_crop.png', './data/train_add_data\\0_0_rotation.png', './data/train_add_data\\0_1_crop.png', './data/train_add_data\\0_1_rotation.png', './data/train_add_data\\0_2_crop.png', './data/train_add_data\\0_2_rotation.png', './data/train_add_data\\0_3_crop.png', './data/train_add_data\\0_3_rotation.png', './data/train_add_data\\0_4_crop.png', './data/train_add_data\\0_4_rotation.png', './data/train_add_data\\0_5_crop.png', './data/train_add_data\\0_5_rotation.png', './data/train_add_data\\0_6_crop.png', './data/train_add_data\\0_6_rotation.png', './data/train_add_data\\0_7_crop.png', './data/train_add_data\\0_7_rotation.png', './data/train_add_data\\0_8_crop.png', './data/train_add_data\\0_8_rotation.png', './data/train_add_data\\0_9_crop.png', './data/train_add_data\\0_9_rotation.png', './data/train_add_data\\0_10_crop.png', './data/train_add_data\\0_10_rotation.png', './data/train_add_data\\0_11_crop.png', './data/train_add_data\\0_11_rotation.png', './data/tra

  5%|████▎                                                                              | 1/19 [00:02<00:41,  2.33s/it]

['./data/train_add_data\\4_0_crop.png', './data/train_add_data\\4_0_rotation.png', './data/train_add_data\\4_1_crop.png', './data/train_add_data\\4_1_rotation.png', './data/train_add_data\\4_2_crop.png', './data/train_add_data\\4_2_rotation.png', './data/train_add_data\\4_3_crop.png', './data/train_add_data\\4_3_rotation.png', './data/train_add_data\\4_4_crop.png', './data/train_add_data\\4_4_rotation.png', './data/train_add_data\\4_5_crop.png', './data/train_add_data\\4_5_rotation.png', './data/train_add_data\\4_6_crop.png', './data/train_add_data\\4_6_rotation.png', './data/train_add_data\\4_7_crop.png', './data/train_add_data\\4_7_rotation.png', './data/train_add_data\\4_8_crop.png', './data/train_add_data\\4_8_rotation.png', './data/train_add_data\\4_9_crop.png', './data/train_add_data\\4_9_rotation.png', './data/train_add_data\\4_10_crop.png', './data/train_add_data\\4_10_rotation.png', './data/train_add_data\\4_11_crop.png', './data/train_add_data\\4_11_rotation.png', './data/tra

 26%|█████████████████████▊                                                             | 5/19 [00:05<00:14,  1.05s/it]

...

 79%|████████████████████████████████████████████████████████████████▋                 | 15/19 [00:54<00:18,  4.65s/it]

['./data/train_add_data\\16_0_crop.png', './data/train_add_data\\16_0_rotation.png', './data/train_add_data\\16_1_crop.png', './data/train_add_data\\16_1_rotation.png', './data/train_add_data\\16_2_crop.png', './data/train_add_data\\16_2_rotation.png', './data/train_add_data\\16_3_crop.png', './data/train_add_data\\16_3_rotation.png', './data/train_add_data\\16_4_crop.png', './data/train_add_data\\16_4_rotation.png', './data/train_add_data\\16_5_rotation.png', './data/train_add_data\\16_6_rotation.png', './data/train_add_data\\16_7_rotation.png', './data/train_add_data\\16_8_rotation.png', './data/train_add_data\\16_9_rotation.png', './train\\틈새과다\\0.png', './train\\틈새과다\\1.png', './train\\틈새과다\\2.png', './train\\틈새과다\\3.png', './train\\틈새과다\\4.png']


 89%|█████████████████████████████████████████████████████████████████████████▎        | 17/19 [00:55<00:06,  3.24s/it]

['./data/train_add_data\\17_0_crop.png', './data/train_add_data\\17_0_rotation.png', './data/train_add_data\\17_1_crop.png', './data/train_add_data\\17_1_rotation.png', './data/train_add_data\\17_2_crop.png', './data/train_add_data\\17_2_rotation.png', './data/train_add_data\\17_3_crop.png', './data/train_add_data\\17_3_rotation.png', './data/train_add_data\\17_4_crop.png', './data/train_add_data\\17_4_rotation.png', './data/train_add_data\\17_5_crop.png', './data/train_add_data\\17_5_rotation.png', './data/train_add_data\\17_6_crop.png', './data/train_add_data\\17_6_rotation.png', './data/train_add_data\\17_7_crop.png', './data/train_add_data\\17_7_rotation.png', './data/train_add_data\\17_8_crop.png', './data/train_add_data\\17_8_rotation.png', './data/train_add_data\\17_9_crop.png', './data/train_add_data\\17_9_rotation.png', './data/train_add_data\\17_10_crop.png', './data/train_add_data\\17_10_rotation.png', './data/train_add_data\\17_11_crop.png', './data/train_add_data\\17_11_ro

100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [01:08<00:00,  3.62s/it]


In [9]:
for label in tqdm(oslabel):
    for i in range(len(short_label)):
        if short_label[i] == label:
            idx = 0
            one_sample = train_df[train_df['label'] == label].reset_index(drop=True)
            images_list = natsorted(one_sample['img_path'])
            print(images_list)
            for _, image_name in enumerate(images_list):
                image = np.array(Image.open(opj(image_name)).convert('RGB'))

                aug_img = cv2.flip(image, 2)
                aug_img = cv2.resize(aug_img, dsize=(1024, 1024))
                aug_img = cv2.cvtColor(aug_img, cv2.COLOR_BGR2RGB)
                save_path = opj('./data/train_add_data', f'{label}_{idx}_flip2.png')
                save_name = f'{label}_{idx}_flip2.png'
                idx += 1
                cv2.imwrite(save_path, aug_img)
                train_df.loc[len(train_df)] = [save_path] + one_sample.iloc[0][1:].values.tolist()   

  0%|                                                                                           | 0/19 [00:00<?, ?it/s]

['./data/train_add_data\\0_0_crop.png', './data/train_add_data\\0_0_flip1.png', './data/train_add_data\\0_0_rotation.png', './data/train_add_data\\0_1_crop.png', './data/train_add_data\\0_1_flip1.png', './data/train_add_data\\0_1_rotation.png', './data/train_add_data\\0_2_crop.png', './data/train_add_data\\0_2_flip1.png', './data/train_add_data\\0_2_rotation.png', './data/train_add_data\\0_3_crop.png', './data/train_add_data\\0_3_flip1.png', './data/train_add_data\\0_3_rotation.png', './data/train_add_data\\0_4_crop.png', './data/train_add_data\\0_4_flip1.png', './data/train_add_data\\0_4_rotation.png', './data/train_add_data\\0_5_crop.png', './data/train_add_data\\0_5_flip1.png', './data/train_add_data\\0_5_rotation.png', './data/train_add_data\\0_6_crop.png', './data/train_add_data\\0_6_flip1.png', './data/train_add_data\\0_6_rotation.png', './data/train_add_data\\0_7_crop.png', './data/train_add_data\\0_7_flip1.png', './data/train_add_data\\0_7_rotation.png', './data/train_add_data\

  5%|████▎                                                                              | 1/19 [00:04<01:25,  4.76s/it]

['./data/train_add_data\\4_0_crop.png', './data/train_add_data\\4_0_flip1.png', './data/train_add_data\\4_0_rotation.png', './data/train_add_data\\4_1_crop.png', './data/train_add_data\\4_1_flip1.png', './data/train_add_data\\4_1_rotation.png', './data/train_add_data\\4_2_crop.png', './data/train_add_data\\4_2_flip1.png', './data/train_add_data\\4_2_rotation.png', './data/train_add_data\\4_3_crop.png', './data/train_add_data\\4_3_flip1.png', './data/train_add_data\\4_3_rotation.png', './data/train_add_data\\4_4_crop.png', './data/train_add_data\\4_4_flip1.png', './data/train_add_data\\4_4_rotation.png', './data/train_add_data\\4_5_crop.png', './data/train_add_data\\4_5_flip1.png', './data/train_add_data\\4_5_rotation.png', './data/train_add_data\\4_6_crop.png', './data/train_add_data\\4_6_flip1.png', './data/train_add_data\\4_6_rotation.png', './data/train_add_data\\4_7_crop.png', './data/train_add_data\\4_7_flip1.png', './data/train_add_data\\4_7_rotation.png', './data/train_add_data\

 26%|█████████████████████▊                                                             | 5/19 [00:11<00:30,  2.18s/it]

...

 79%|████████████████████████████████████████████████████████████████▋                 | 15/19 [01:27<00:28,  7.21s/it]

['./data/train_add_data\\16_0_crop.png', './data/train_add_data\\16_0_flip1.png', './data/train_add_data\\16_0_rotation.png', './data/train_add_data\\16_1_crop.png', './data/train_add_data\\16_1_flip1.png', './data/train_add_data\\16_1_rotation.png', './data/train_add_data\\16_2_crop.png', './data/train_add_data\\16_2_flip1.png', './data/train_add_data\\16_2_rotation.png', './data/train_add_data\\16_3_crop.png', './data/train_add_data\\16_3_flip1.png', './data/train_add_data\\16_3_rotation.png', './data/train_add_data\\16_4_crop.png', './data/train_add_data\\16_4_flip1.png', './data/train_add_data\\16_4_rotation.png', './data/train_add_data\\16_5_flip1.png', './data/train_add_data\\16_5_rotation.png', './data/train_add_data\\16_6_flip1.png', './data/train_add_data\\16_6_rotation.png', './data/train_add_data\\16_7_flip1.png', './data/train_add_data\\16_7_rotation.png', './data/train_add_data\\16_8_flip1.png', './data/train_add_data\\16_8_rotation.png', './data/train_add_data\\16_9_flip1

 89%|█████████████████████████████████████████████████████████████████████████▎        | 17/19 [01:29<00:09,  4.98s/it]

['./data/train_add_data\\17_0_crop.png', './data/train_add_data\\17_0_flip1.png', './data/train_add_data\\17_0_rotation.png', './data/train_add_data\\17_1_crop.png', './data/train_add_data\\17_1_flip1.png', './data/train_add_data\\17_1_rotation.png', './data/train_add_data\\17_2_crop.png', './data/train_add_data\\17_2_flip1.png', './data/train_add_data\\17_2_rotation.png', './data/train_add_data\\17_3_crop.png', './data/train_add_data\\17_3_flip1.png', './data/train_add_data\\17_3_rotation.png', './data/train_add_data\\17_4_crop.png', './data/train_add_data\\17_4_flip1.png', './data/train_add_data\\17_4_rotation.png', './data/train_add_data\\17_5_crop.png', './data/train_add_data\\17_5_flip1.png', './data/train_add_data\\17_5_rotation.png', './data/train_add_data\\17_6_crop.png', './data/train_add_data\\17_6_flip1.png', './data/train_add_data\\17_6_rotation.png', './data/train_add_data\\17_7_crop.png', './data/train_add_data\\17_7_flip1.png', './data/train_add_data\\17_7_rotation.png',

100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [01:52<00:00,  5.90s/it]


# label 인코딩 후 증강된 csv 저장

In [10]:
train_labels = train_df["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]
train_labels

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [11]:
train_df['encoder_label'] = train_labels

In [12]:
train_df.to_csv('./data/train_df_add_data.csv', index=False, encoding="utf-8-sig")

# 증강된 데이터 셋 불러오기

In [14]:
DATA_DIR = './data'

train_df = pd.read_csv(os.path.join(DATA_DIR, 'train_df_add_data.csv'))
test_df = pd.read_csv(os.path.join(DATA_DIR, 'test.csv'))

print(train_df.head())
print(test_df.head())
print(train_df.shape)
print(test_df.shape)

              img_path  label description  encoder_label
0   ./train\가구수정\0.png      0        가구수정              0
1   ./train\가구수정\1.png      0        가구수정              0
2  ./train\가구수정\10.png      0        가구수정              0
3  ./train\가구수정\11.png      0        가구수정              0
4   ./train\가구수정\2.png      0        가구수정              0
         id        img_path
0  TEST_000  ./test/000.png
1  TEST_001  ./test/001.png
2  TEST_002  ./test/002.png
3  TEST_003  ./test/003.png
4  TEST_004  ./test/004.png
(7387, 4)
(792, 2)


In [15]:
train_df['label'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18], dtype=int64)

In [16]:
class_num = len(train_df.encoder_label.unique())
class_num

19

In [17]:
train_df['label']

0        0
1        0
2        0
3        0
4        0
        ..
7382    17
7383    17
7384    17
7385    17
7386    17
Name: label, Length: 7387, dtype: int64